# 不停弹出TF框架 那就手动依次执行代码 等待转圈结束再执行下一个 这样的话检测几率会小一点

# 简介

>  原百度AI项目地址（B站id:xiaoxu12315）：[点我](https://aistudio.baidu.com/aistudio/projectdetail/745434)

>Github作者:[我](https://github.com/AliaksandrSiarohin/first-order-model)还有[我](https://github.com/dunnousername/yanderifier)

>  B站教程：[点我](https://www.bilibili.com/video/BV1Mv41117LV)

> 此版本为匹配B站视频教学，基于xiaoxu代码修改，对使用有疑问请加QQ群：776823471

**为减少高峰期，在不使用的时候务必到项目首页点击停止**

**嘛...搞出来自己玩玩或者给朋友发就行了...还是不要传到b站为好**

**QQ群：776823471 提供更多口型素材 并负责手把手教学 入群免费代做一次**
![](https://ai-studio-static-online.cdn.bcebos.com/1a8186bc7a864faf84084ed222615a11b637be72f9fd4a6b90b7b3e785a3f59a)
**部分素材效果演示：**[点我](http://www.bilibili.com/video/BV1z54y127fB)


>**在file文件夹内放素材,将视频更改为import.mp4,图片更改为import.png 视频和图片分辨率:256x256 

有时候刷不出file文件夹，自己新建一个就可以了，新建按钮在上传按钮的左边，第二次做，只需要删除要换的素材后上传新的就可以了。

# **最佳效果方案：**

尽量只保留脖子到头发的部分，其他地方不留；图片人物脸的角度参照视频母体的人物角度，建议使用真人，非三次元人类做出来效果会怪怪的

**例如：unravel母体的最佳图片是证件照的标准。**

![](https://ai-studio-static-online.cdn.bcebos.com/5f486a61622145c49968c07ae2f2c366909f544f105e46aabcb112d6c7c750e9)
![](https://ai-studio-static-online.cdn.bcebos.com/0986c2b958ce44439592d952276ad4ced9362c201d1840ceb6dbd91a69df9597)
![](https://ai-studio-static-online.cdn.bcebos.com/a84dc5dafd314ed2ac657df4bf7d7881deef718b468c4f8f83c645c16a097e69)

![](https://ai-studio-static-online.cdn.bcebos.com/b6e3aad4214f4d71a153e079c4729d37e6693230fc334879ba603e130442943b)
![](https://ai-studio-static-online.cdn.bcebos.com/07d6e2bd8c1146e5b421d43d24b400671e1cdd092c704239b874fe1ac031af17)
![](https://ai-studio-static-online.cdn.bcebos.com/746437a626754f07bc6012e7b58d758d1f59af5e5e0c4f6088530d3cd205a8bc)



**使用过程中遇到以下提示,重启项目便可**

![](https://ai-studio-static-online.cdn.bcebos.com/8e2c76e9c4274c3ba34b266e0d1f0f9fe6c13581515c4b5b9cbc491caa4c4d4e)

若不小心误动代码，请重置项目：[点我](https://aistudio.baidu.com/aistudio/projectdetail/986206) 进去后重新运行一次即可




# 代码部分

In [1]:
#查看数据集
!ls /home/aistudio/data

data51886


In [2]:
!ls /home/aistudio/work

In [3]:
import sys
sys.path.append('/home/aistudio/external-libraries')
sys.path.append('/home/aistudio/first-order-model')

In [4]:
!git clone https://gitee.com/xiaoxu12315/first-order-model.git
#从码云克隆仓库
!cd /home/aistudio/first-order-model
#进入工作目录
#如果这一步出错可以更换其他源 https://blog.csdn.net/sinat_21591675/article/details/82770360
#红色报错代码：ERROR: visualdl 2.0.0b8 has requirement Pillow>=7.0.0, but you'll have pillow 5.2.0 which is incompatible. 可以无视 不影响程序执行
#!pip install -i https://mirrors.aliyun.com/pypi/simple/ moviepy cffi==1.11.5 cloudpickle==1.2.1 cycler==0.10.0 dask==0.18.2 decorator==4.3.0 imageio==2.5.0 kiwisolver==1.0.1 matplotlib==2.2.2 networkx==2.1 numpy==1.15.0 pandas==0.23.4 Pillow==5.2.0 pycparser==2.18 pygit==0.1 pyparsing==2.2.0 python-dateutil==2.7.3 pytz==2018.5 PyWavelets==0.5.2 PyYAML==5.1 scikit-image==0.14.0 scikit-learn==0.19.2 scipy==1.1.0 six==1.14.0 toolz==0.9.0 torch==1.0.0 torchvision==0.2.1 tqdm==4.24.0
!pip install -i https://mirrors.aliyun.com/pypi/simple/ imageio-ffmpeg scikit-image torch==1.0.0 torchvision==0.2.1
#安装依赖

Cloning into 'first-order-model'...
remote: Enumerating objects: 243, done.
remote: Counting objects: 100% (243/243), done.
remote: Compressing objects: 100% (123/123), done.


In [5]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('file/import.png')
driving_video = imageio.mimread('file/import.mp4',memtest=False)


#调整图片和视频分辨率

source_image = resize(source_image, (256, 256))[..., :3]
driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, driving_video).to_html5_video())

In [6]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='first-order-model/config/vox-256.yaml', 
                            checkpoint_path='data/data51886/checkpoint.tar')
                            #加载配置文件并加载数据集

In [7]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#保存结果
imageio.mimsave('work/result 1.mp4', [img_as_ubyte(frame) for frame in predictions])

HTML(display(source_image, driving_video, predictions).to_html5_video())

In [8]:
import cv2
from cv2 import VideoWriter,VideoWriter_fourcc

video_root ="file/import.mp4"
cap = cv2.VideoCapture(video_root)
fps = cap.get(cv2.CAP_PROP_FPS)
video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
resize_width = 256
resize_height = 256
double_fps = 2*fps

In [9]:
import cv2
from cv2 import VideoWriter,VideoWriter_fourcc

video_root ="work/result 1.mp4"
out_root = "work/result 2.mp4"
cap = cv2.VideoCapture(video_root)
resize_width = 256
resize_height = 256
double_fps = 2*fps

fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
videoWriter = cv2.VideoWriter(out_root,fourcc,fps,(256,256))

success = cap.isOpened()
while (success):
    success,frame = cap.read()
    if not success:
        break
    frame = cv2.resize(frame,(256,256))
    videoWriter.write(frame)
videoWriter.release()

In [10]:
from moviepy.editor import *

# 读取2个视频文件 
videoclip_1 = VideoFileClip("file/import.mp4")
videoclip_2 = VideoFileClip("work/result 2.mp4")

# 提取第一个视频文件的音频部分
audio_1 = videoclip_1.audio

# 将提取的音频和第二个视频文件进行合成
videoclip_3 = videoclip_2.set_audio(audio_1)
# 输出新的视频文件
videoclip_3.write_videofile("file/export.mp4", audio_codec="aac")


![](https://ai-studio-static-online.cdn.bcebos.com/2541267cbea94a4882f7c5baacc40e5fd13384954996462fa6f6feb510d6b01e)
<center><font size="10" >许可证:<a target="_blank" href="https://creativecommons.org/licenses/by-nc/4.0/">CC-BY-NC v4.0</a></font></center>


